In [1]:
import pybel
import os
import logging
import nx2d3
import py2neo
import json
import time

In [2]:
logging.basicConfig(level=logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)

In [3]:
time.asctime()

'Sat Oct  8 20:19:10 2016'

In [4]:
base = os.environ['PYBEL_BASE']
path = os.path.join(base, 'tests', 'bel', 'test_bel_1.bel')

In [5]:
g = pybel.from_path(path)

#neo_graph = Graph('')
#g.to_neo4j(neo_graph)

INFO:pybel.parser.utils:File length: 26. Document section until line 7, definition section until line 12
INFO:pybel.graph:Finished parsing document section in 0.01 seconds
INFO:root:Downloading http://resource.belframework.org/belframework/1.0/namespace/chebi-names.belns
INFO:root:Downloading http://resource.belframework.org/belframework/1.0/annotation/mesh-cell-structure.belanno
INFO:pybel.graph:Finished parsing definitions section in 1.80 seconds
ERROR:pybel.graph:PyBEL0018 Nested statements not supported. Please explicitly specifiy. p(ALPHABET:A) -> (p(ALPHABET:B) -> p(ALPHABET:C))
INFO:pybel.graph:Finished parsing statements section in 51.38 seconds


In [6]:
time.asctime()

'Sat Oct  8 20:20:03 2016'

In [7]:
nx2d3.embed_networkx(g, height=300, width=600)

<IPython.core.display.Javascript object>

In [8]:
print_edges = True
if print_edges:
    for u,v,data in g.edges(data=True):
        print(u, v, json.dumps(data, indent=2))

('RNA', 'ALPHABET', 'D') ('Protein', 'ALPHABET', 'D') {
  "relation": "translatedTo"
}
('Gene', 'ALPHABET', 'E') ('RNA', 'ALPHABET', 'E') {
  "relation": "transcribedTo"
}
('Gene', 'ALPHABET', 'D') ('RNA', 'ALPHABET', 'D') {
  "relation": "transcribedTo"
}
('RNA', 'ALPHABET', 'C') ('Protein', 'ALPHABET', 'C') {
  "relation": "translatedTo"
}
('Gene', 'ALPHABET', 'C') ('RNA', 'ALPHABET', 'C') {
  "relation": "transcribedTo"
}
('RNA', 'ALPHABET', 'B') ('Protein', 'ALPHABET', 'B') {
  "relation": "translatedTo"
}
('Gene', 'ALPHABET', 'B') ('RNA', 'ALPHABET', 'B') {
  "relation": "transcribedTo"
}
('Protein', 'ALPHABET', 'B') ('Protein', 'ALPHABET', 'D') {
  "citation_name": "That one article from last week",
  "TESTAN2": "3",
  "citation_type": "Pubmed",
  "citation_reference": "123455",
  "relation": "directlyDecreases",
  "Evidence": "Evidence 2"
}
('Protein', 'ALPHABET', 'B') ('Protein', 'ALPHABET', 'C') {
  "citation_name": "That one article from last week",
  "TESTAN2": "3",
  "citat

In [9]:
neo_path = os.environ['NEO_PATH']
neo = py2neo.Graph(neo_path)
neo.data('match (n) detach delete n')

INFO:httpstream:> GET http://localhost:7474/db/data/
ERROR:httpstream:! SocketError: Connection refused


SocketError: Connection refused

In [ ]:
g.to_neo4j(neo)

In [ ]:
neo.data('match (n) return n')

In [ ]:
r = neo.run('match (n)-[k]-(m) return n.namespace, n.value, k.relation, m.namespace, m.value')
r.dump()